# Detecton of sugarbeet plants with YOLO11n and Cross Validation

## I Daten Vorbereiten

### Ziel
Dieser Codeabschnitt erstellt die Datenaufteilung für ein k-fold Cross-Validation-Szenario. Die Bild- und Labeldateien werden zufällig in Trainings- und Validierungssplits unterteilt und in einer klar definierten Verzeichnisstruktur organisiert.

### Beschreibung

#### 1. Initialisierung
- **Bildverzeichnis**: `datensatz_sugarbeet/img`, enthält die Bilddateien.
- **Labelverzeichnis**: `datensatz_sugarbeet/convertedSugarbeetDataset`, enthält die zugehörigen Labeldateien.
- **Ausgabeverzeichnis**: `datensatz_sugarbeet_Yolo/kfold_split`, Zielverzeichnis für die Splits.
- **Anzahl der Folds**: 5.

#### 2. Funktion `create_kfold_splits`
Die Funktion übernimmt die Erstellung der k-fold Cross-Validation-Splits:
1. **Zufällige Aufteilung**:
   - Alle Bilddateien im Bildverzeichnis werden zufällig sortiert.
   - Die Gesamtanzahl der Bilder wird gleichmäßig auf die Folds aufgeteilt.
2. **Verzeichnisstruktur**:
   - Für jeden Fold wird eine Ordnerstruktur erstellt, die Trainings- und Validierungsdaten getrennt speichert.
3. **Datenverschiebung**:
   - Bilder und zugehörige Labels werden in die entsprechenden Unterordner kopiert:
     - **Trainingsdaten**: In `train/images` und `train/labels`.
     - **Validierungsdaten**: In `val/images` und `val/labels`.

#### 3. Ergebnisse
Nach Abschluss des Prozesses enthält das Zielverzeichnis die k-fold Aufteilung der Daten, die für das Training und die Validierung verwendet werden können.


In [4]:
import os
import shutil
import random


# Define directories
image_dir = "datensatz_sugarbeet/img"
label_dir = "datensatz_sugarbeet/annotations"
output_dir = "datensatz_sugarbeet/cross_validation"

k_folds = 5

def create_kfold_splits(image_dir, label_dir, output_dir, k_folds):
    """
    Erstellt k-Fold Cross-Validation Splits.
    
    Parameters:
        image_dir: str  - Pfad zu den Bilddateien
        label_dir: str  - Pfad zu den Labeldateien
        output_dir: str - Pfad zum Ausgabeverzeichnis
        k_folds: int    - Anzahl der Folds
    """
    images = [f for f in os.listdir(image_dir) if f.endswith((".png", ".jpg", ".jpeg"))]
    random.shuffle(images)  # Zufällige Reihenfolge
    num_images = len(images)
    fold_size = num_images // k_folds

    for fold in range(k_folds):
        # Erstelle Ordnerstruktur für jeden Fold
        fold_dir = os.path.join(output_dir, f"fold_{fold}")
        os.makedirs(f"{fold_dir}/train/images", exist_ok=True)
        os.makedirs(f"{fold_dir}/train/labels", exist_ok=True)
        os.makedirs(f"{fold_dir}/val/images", exist_ok=True)
        os.makedirs(f"{fold_dir}/val/labels", exist_ok=True)

        val_images = images[fold * fold_size : (fold + 1) * fold_size]
        train_images = [img for img in images if img not in val_images]

        # Verschiebe Trainings- und Validierungsdaten in die entsprechenden Ordner
        for img_set, img_dir, label_dir_target in [
            (train_images, f"{fold_dir}/train/images", f"{fold_dir}/train/labels"),
            (val_images, f"{fold_dir}/val/images", f"{fold_dir}/val/labels"),
        ]:
            for image_file in img_set:
                # Kopiere Bilddatei
                src_img_path = os.path.join(image_dir, image_file)
                dst_img_path = os.path.join(img_dir, image_file)
                shutil.copy(src_img_path, dst_img_path)

                # Kopiere zugehörige Labeldatei
                label_file = os.path.splitext(image_file)[0] + ".txt"
                src_label_path = os.path.join(label_dir, label_file)
                if os.path.exists(src_label_path):
                    dst_label_path = os.path.join(label_dir_target, label_file)
                    shutil.copy(src_label_path, dst_label_path)

    print(f"{k_folds}-fold Cross-Validation Splits erstellt!")

if __name__ == "__main__":
    create_kfold_splits(image_dir, label_dir, output_dir, k_folds)


5-fold Cross-Validation Splits erstellt!


### 

## II k-Fold Modelle trainieren

### Ziel
Dieser Codeabschnitt dient dazu, ein YOLO-Modell mithilfe von k-fold Cross-Validation zu trainieren. Dabei werden die verfügbaren Daten in Trainings- und Validierungssets aufgeteilt, um die Performance des Modells robust zu evaluieren.

### Beschreibung

#### 1. Initialisierung
- **Modellpfad**: `yolo11n.pt`, das YOLO-Modell, das trainiert wird.
- **Anzahl der Folds**: 5.
- **Ausgabeverzeichnis**: `datensatz_sugarbeet_Yolo/kfold_split`, wo die trainierten Modelle und Zwischenergebnisse gespeichert werden.

#### 2. YAML-Datei-Erstellung
- Die Funktion `create_yaml_file` generiert eine YAML-Datei, die YOLO die Pfade zu den Trainings- und Validierungsdaten sowie die Klassennamen bereitstellt.

#### 3. Training mit k-fold Cross-Validation
- Die Funktion `train_with_kfold` übernimmt das Training für jeden Fold:
  - **Schritt 1**: Verzeichnisse für Trainings- und Validierungsdaten des jeweiligen Folds definieren.
  - **Schritt 2**: Eine YAML-Datei für den Fold erstellen.
  - **Schritt 3**: Das YOLO-Modell laden und trainieren.
  - **Schritt 4**: Metriken wie `mAP_50`, `Precision` und `Recall` extrahieren und in einer Ergebnistabelle speichern.

#### 4. Ergebnisse
Nach Abschluss des Trainings wird eine Übersicht der Metriken für jeden Fold in `results_summary` ausgegeben.



In [5]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device: ", device)

Device:  cpu


In [ ]:
import yaml
from ultralytics import YOLO

# YOLO Model
model_path = "yolo11n.pt"
output_dir = "datensatz_sugarbeet_Yolo\kfold_split"
k_folds = 5
def create_yaml_file(train_dir, val_dir, yaml_path):
    """
    Erstellt eine YAML-Datei für YOLO-Training mit korrekten absoluten Pfaden.
    
    Parameters:
        train_dir: str - Pfad zum Trainingsdatensatz
        val_dir: str   - Pfad zum Validierungsdatensatz
        yaml_path: str - Pfad zur YAML-Datei
    """
    # Konvertiere train_dir und val_dir in absolute Pfade
    train_dir = os.path.abspath(train_dir)
    val_dir = os.path.abspath(val_dir)
    
    data = {
        "train": train_dir,
        "val": val_dir,
        "names": ["sugarbeet"]#, "weed", "rock"]  # Klassen
    }
    os.makedirs(os.path.dirname(yaml_path), exist_ok=True)
    with open(yaml_path, "w") as f:
        yaml.dump(data, f)


def train_with_kfold(k_folds, model_path, output_dir):
    results_summary = {}

    for fold in range(k_folds):
        print(f"Training Fold {fold + 1}/{k_folds}...")

        fold_dir = os.path.join(output_dir, f"fold_{fold}")
        train_data = f"{fold_dir}/train/images"
        val_data = f"{fold_dir}/val/images"

        yaml_path = os.path.join(fold_dir, "dataset.yaml")
        print(os.path.abspath(yaml_path))
        create_yaml_file(train_data, val_data, yaml_path)

        model = YOLO(model_path)

        results = model.train(
            data=yaml_path,
            epochs=1,
            imgsz=640
        )

        # Extrahiere die Metriken
        mean_results = results.box.mean_results()
        val_mAP50 = mean_results[2]
        precision = mean_results[0]
        recall = mean_results[1]

        results_summary[f"Fold_{fold + 1}"] = {
            "val_mAP50": val_mAP50,
            "precision": precision,
            "recall": recall,
        }
        print(f"Training Fold {fold + 1} abgeschlossen!")

    return results_summary


if __name__ == "__main__":
    results_summary = train_with_kfold(k_folds, model_path, output_dir)
    print("Training mit k-fold abgeschlossen!")
    print(results_summary)



## III Cross-Validation Ergebnisse Zusammenfassen

### Ziel
Der Code analysiert die Ergebnisse der k-fold Cross-Validation aus den generierten `results.csv`-Dateien, extrahiert die relevanten Metriken (Precision, Recall, mAP_50) und speichert eine Zusammenfassung in einer Datei `crossValidation_summary.txt`.

### Beschreibung
- **Eingaben**:
  - Ergebnisse der Cross-Validation werden in `results.csv`-Dateien gespeichert, die sich in den Verzeichnissen `runs/detect/train<fold_number>` befinden.
  - Jede CSV-Datei enthält verschiedene Metriken und Loss-Werte pro Epoche.

- **Verwendete Metriken**:
  - `mAP_50` → Spaltenname: `metrics/mAP50(B)`
  - `Precision` → Spaltenname: `metrics/precision(B)`
  - `Recall` → Spaltenname: `metrics/recall(B)`

- **Ausgabe**:
  - Eine Textdatei `crossValidation_summary.txt`, die die Metriken für jeden Fold zusammenfasst.


In [ ]:
import os
import csv

# Basisverzeichnis für die k-fold Ergebnisse
output_dir = "runs/detect"
summary_file = os.path.join(output_dir, "crossValidation_summary.txt")

# Metriken und deren zugehörige Spaltennamen in der CSV-Datei
metrics_mapping = {
    "mAP_50": "metrics/mAP50(B)",
    "Precision": "metrics/precision(B)",
    "Recall": "metrics/recall(B)"
}

summary = {}

for fold in range(5):
    result_path = os.path.join(output_dir, f"train{fold}", "results.csv")  # Passe den Pfad an, falls nötig
    if os.path.exists(result_path):
        fold_metrics = {metric: None for metric in metrics_mapping.keys()}
        with open(result_path, "r") as csvfile:
            reader = csv.DictReader(csvfile)  # Verwende DictReader für Spaltennamen
            for row in reader:
                for metric, column_name in metrics_mapping.items():
                    if column_name in row:
                        fold_metrics[metric] = row[column_name]
        summary[f"Fold {fold+1}"] = fold_metrics
    else:
        print(f"Keine Ergebnisse für Fold {fold+1} gefunden.")

# Schreibe Zusammenfassung
with open(summary_file, "w") as f:
    for fold, metrics in summary.items():
        f.write(f"{fold}:\n")
        for metric, value in metrics.items():
            f.write(f"  {metric}: {value}\n")
        f.write("\n")

print(f"Zusammenfassung gespeichert: {summary_file}")


Zusammenfassung gespeichert: runs/detect\crossValidation_summary.txt


## IV Detect sugarbeets with best model

- replace source with image or directory of images
- model saves copy of pictures with bounding boxes in runs\detect\predict
- prints predected bounding boxes
- plot image with sugarbeet predictions

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the trained model
model = YOLO("runs/detect/train16/weights/best.pt")
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
results = model.predict(source=os.path.join(parent_dir,"imageSet/img/sugarbeet_bbch12_000001.png"), save=True, imgsz=640)

# Access predictions
for result in results:
    boxes = result.boxes.xyxy.numpy()  # all Bounding-Box-coordinates
    classes = result.boxes.cls.numpy()  # class-IDs
    sugarbeet_boxes = boxes[classes == 0] # all bounding boxes of sugarbeets
    print("Sugarbeet Boxes:", sugarbeet_boxes)

# plot image with predicted boundingboxes

image = mpimg.imread('runs/detect/predict/sugarbeet_bbch12_000001.jpg')
plt.imshow(image)
plt.axis('off')
plt.show()
